# Accessing  HEASARC tables through the TAP with ADQL

We have used the __[Table Access Protocol](http://www.ivoa.net/documents/TAP/)__ (TAP) protocol in several other notebooks for basic queries.  Here, we expand on its usage and that of the __[Astronomical Data Query Language](http://www.ivoa.net/documents/latest/ADQL.html)__ (ADQL) that it uses.  

* [1. Basic](#basic) Table Access Protocol queries
* [2. Cross-correlating](#cc) our own catalog with a HEASARC catalog
* [3. Combining](#combo) data from multiple catalogs and cross-correlating


In [1]:
import numpy
## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")
## The main HTTP request tool we will use:
import requests
## For simple astropy tables
import astropy, io

## For handling ordinary astropy Tables
from astropy.table import Table

## For handling VO table type objects
from astropy.io import votable as apvot

<a id="basic"></a>

# 1. Basic Table Access Protocol queries

A TAP query is the most powerful way to search a catalog.  

A Simple Cone Search only allows you to ask for a position and radius:  

In [13]:
import astropy.coordinates as coord
coord=coord.SkyCoord.from_name("m51")
print(coord)
params = {'table': 'zcat', 'RA':coord.ra.deg , 'DEC':coord.dec.deg, 'SR':1}
r = requests.get('https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl', params=params)
table=Table.read(io.BytesIO(r.content))
table.show_in_notebook()

<SkyCoord (ICRS): (ra, dec) in deg
    ( 202.484167,  47.230556)>


idx,name,ra,dec,bmag,radial_velocity,radial_velocity_error,redshift,class,Search_Offset
,,deg,deg,,km / s,km / s,,,
0,13267+4631,202.20650,46.25850,15.10,0,0,--,9999,59.4301
1,N5173,202.10529,46.59082,14.12,2467,26,--,6300,41.4067
2,N5198,202.54745,46.66996,13.30,2569,25,--,6300,33.7355
3,N5169,202.04237,46.67154,14.70,2482,40,--,6200,38.1107
4,I4263,202.13883,46.92671,15.40,2663,25,--,6200,23.0534
5,N5194,202.46823,47.19815,9.03,474,23,--,6200,2.0500
6,N5195,202.49491,47.26792,10.94,558,23,--,6200,2.2841
7,1325+4754,201.87771,47.64124,--,18107,179,--,9999,34.8278
8,1325+4754,201.95234,47.64138,--,18296,11,--,9999,32.7636


With the TAP, you can refine the search based on any other attribute in the given catalog.  

The basics of ADQL:

* *SELECT &#42; FROM catalog as cat* says you want all ("&#42;") columns from the catalog called "catalog", which you will refer to below by the more compact name of "cat", 
* *WHERE cat.bmag < 14* says that you want to retrieve only those entries in the catalog whose bmag column has a value less than 14

There are many other options.  Instead of returning all columns, you can *SELECT cat.RA, cat.DEC, cat.bmag from catalog as cat...* to only return the columns you're interested in.

You can also append *ORDER by cat.bmag* to return the result sorted ascending by one of the columns, adding *DESC* to the end for descending. 

A few special functions in the ADQL allow you to query regions:

* *WHERE contains( point('ICRS', cat.ra, cat.dec), circle('ICRS', 210.5, -6.5, 0.5))=1*

is how you would ask for any catalog entries whose RA,DEC lie within a circular region defined by RA,DEC 210.5,-6.5 and a radius of 0.5 (all in degrees).  The 'ICRS' specifies the coordinate system.  

See the ADQL documentation for more.

With these basics, we do the following:

In [14]:
## These parameters are defined in the TAP standard.  
tap_params = {
    "request":"doQuery",  # for requests, specify the request type
    "lang":"ADQL",        # the language
    "query":              # and the query expressed in that language
    """SELECT ra, dec, Radial_Velocity FROM zcat as cat where 
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',{0},{1},{2}))=1 and
    cat.bmag < 14
    order by cat.radial_velocity_error
    """.format(coord.ra.deg,coord.dec.deg,1.0)
    }
r = requests.get('https://heasarc.gsfc.nasa.gov/xamin/vo/tap/sync', params=tap_params)
#r.content
table=Table.read(io.BytesIO(r.content))
table
#print(tap_params["query"])

ra,dec,radial_velocity
float64,float64,int32
202.46823226999999,47.198148719999999,474
202.49490508,47.267920400000001,558
202.54744757,46.669958979999997,2569


If you aren't sure what columns are available for a given catalog, get all attributes of one row:

In [23]:
## These parameters are defined in the TAP standard.  
cat = 'zcat'
tap_params = {
    "request":"doQuery",  # for requests, specify the request type
    "lang":"ADQL",        # the language
    "query":              # and the query expressed in that language
    "SELECT top 1 * FROM {cat}".format(cat=cat)
    }
r = requests.get('https://heasarc.gsfc.nasa.gov/xamin/vo/tap/sync', params=tap_params)
#r.content
table=Table.read(io.BytesIO(r.content))
print("Colums for {cat}:\n".format(cat=cat))
print(' '.join(table.colnames))

Colums for zcat:

__row name ra dec lii bii bmag radial_velocity radial_velocity_error ref_bmag ref_radial_velocity morph_type bar_type luminosity_class structure diameter_1 diameter_2 bt_mag ugc_or_eso distance rfn_number comments redshift ref_redshift notes class __x_ra_dec __y_ra_dec __z_ra_dec


<a id="cc"></a>
# 2. TAP:  Using the TAP to cross-correlate our objects with a catalog

Now to search all of our sources in one go, we need to upload our own table and do a 'cross-correlation' with the *zcat* table. For more on creating VOTable objects, see that notebook.  Here, we just read one in:  

(This may take awhile - 20 or more seconds) 

In [ ]:
files={'uplt':open('../my_sources.xml', 'rb')}

cc_params={
    'lang': 'ADQL', 
    'request': 'doQuery',
    'upload':'mysources,param:uplt'
    }

cc_params["query"]="""
    SELECT cat.ra, cat.dec, Radial_Velocity 
    FROM zcat cat, tap_upload.mysources mt 
    WHERE
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',mt.ra,mt.dec,0.1))=1
    and Radial_Velocity > 0
    ORDER by cat.ra"""
r = requests.post('https://heasarc.gsfc.nasa.gov/xamin/vo/tap/sync',data=cc_params,stream=True,files=files)
mytable=Table.read(io.BytesIO(r.content))

In [28]:
mytable['ra'].format='.4f'
mytable['dec'].format='.4f'
mytable.show_in_notebook(display_length=6)

idx,ra,dec,radial_velocity
0,19.0684,46.7400,5081
1,19.0864,46.7472,5188
2,20.3348,40.4877,5859
3,125.9044,21.3382,5219
4,125.9044,21.3382,5364
5,135.9942,21.9010,3157
6,136.0007,21.9679,3093
7,141.0915,40.6838,8267
8,146.7033,22.0183,7446
9,146.7033,22.0183,7597


<a id="combo"></a>

# 3.  Combining data from different catalogs and cross-correlating
Now we'd like to take the redshift information (above, as a radial velocity) and determine a search radius to use for each galaxy based on its distance, so that we are are searching within a given physical distance. 

In [35]:
## The radial_velocity is in km/s, and this is just c*z, so
c=3.0e5 # km/s
redshifts=mytable['radial_velocity'].filled(0.)/c  # Filling masked values with zero
mytable['redshift']=redshifts
from astropy import units
physdist=0.05*units.Mpc # 50 kpc physical distance

## This needs scipy.  
from astropy.cosmology import Planck15
angDdist=Planck15.angular_diameter_distance(mytable['redshift'])
## angDdist is returned from the astropy.cosmology module as a Quantity object, 
##  i.e. a value and a unit.  Arctan is smart enough not to operate on quantities
##  that aren't unitless.  So angDdist.value to just get the value.
angDrad=numpy.arctan(physdist/angDdist)
angDdeg=angDrad.to(units.arcmin)
mytable['angDdeg']=angDdeg
mytable['redshift'].format = ".3f"
mytable['angDdeg'].format = ".3f"
mytable.show_in_notebook(display_length=6)

idx,ra,dec,radial_velocity,redshift,angDdeg
,,,,,arcmin
0,19.0684,46.7400,5081,0.017,2.341
1,19.0864,46.7472,5188,0.017,2.294
2,20.3348,40.4877,5859,0.020,2.037
3,125.9044,21.3382,5219,0.017,2.281
4,125.9044,21.3382,5364,0.018,2.220
5,135.9942,21.9010,3157,0.011,3.739
6,136.0007,21.9679,3093,0.010,3.815
7,141.0915,40.6838,8267,0.028,1.458
8,146.7033,22.0183,7446,0.025,1.613


This time, rather than write the table to disk, we'll keep it in memory and give requests a "file-like" object using io.BytesIO():

In [36]:
## In memory only, use an IO stream. 
vot_obj=io.BytesIO()
print(mytable.columns)
apvot.writeto(apvot.from_table(mytable),vot_obj)
## (Reset the "file-like" object to the beginning.)
vot_obj.seek(0)
## 'uplt' is what we'll call it (for 'upload table') 
##   in the requests parameters below, or what you will:
files={'uplt':vot_obj}


<TableColumns names=('ra','dec','radial_velocity','redshift','angDdeg')>


This takes half a minute:

In [ ]:
cc_params={
    'lang': 'ADQL', 
    'request': 'doQuery',
    'upload':'mytable,param:uplt'
    }
## This is your ADQL query, where "mytable" here has to 
##  match what you specified in the upload parameter above
cc_params["query"]="""
    SELECT cat.ra, cat.dec, cat.Radial_Velocity 
    FROM zcat cat, tap_upload.mytable mt 
    WHERE
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',mt.ra,mt.dec,mt.angDdeg))=1
    and cat.Radial_Velocity > 0
    ORDER by cat.ra"""
## The name you give here (tab1) matches what's in the cc_params
r = requests.post('https://heasarc.gsfc.nasa.gov/xamin/vo/tap/sync',data=cc_params,stream=True,files=files)

In [39]:
mytable=Table.read(io.BytesIO(r.content))
mytable['ra'].format='.4f'
mytable['dec'].format='.4f'
mytable.show_in_notebook(display_length=5)

idx,ra,dec,radial_velocity
0,19.0684,46.7400,5081
1,19.0684,46.7400,5081
2,19.0864,46.7472,5188
3,19.0864,46.7472,5188
4,19.1092,39.0525,9830
5,20.3348,40.4877,5859
6,21.6497,39.8760,2610
7,22.0852,39.4886,8133
8,22.3012,39.4262,8263
9,22.5001,40.9737,2807
